In [3]:
import os
import numpy as np
import pandas as pd
import json
import shutil

from copy import deepcopy


In [4]:
df=pd.read_csv("train_labels.csv")

# to list
labels= df[df["Motor axis 0"]!=-1].values
len(labels)

451

In [39]:
df.head(5)

,row_id,tomo_id,Motor axis 0,Motor axis 1,Motor axis 2,Array shape (axis 0),Array shape (axis 1),Array shape (axis 2),Voxel spacing,Number of motors
0,0,tomo_003acc,-1.0,-1.0,-1.0,500,1912,1847,6.5,0
1,1,tomo_00e047,169.0,546.0,603.0,300,959,928,15.6,1
2,2,tomo_00e463,235.0,403.0,137.0,500,924,956,19.7,6
3,3,tomo_00e463,243.0,363.0,153.0,500,924,956,19.7,6
4,4,tomo_00e463,222.0,379.0,144.0,500,924,956,19.7,6


In [48]:
df[df["Array shape (axis 0)"]>499]

,row_id,tomo_id,Motor axis 0,Motor axis 1,Motor axis 2,Array shape (axis 0),Array shape (axis 1),Array shape (axis 2),Voxel spacing,Number of motors
0,0,tomo_003acc,-1.0,-1.0,-1.0,500,1912,1847,6.5,0
2,2,tomo_00e463,235.0,403.0,137.0,500,924,956,19.7,6
3,3,tomo_00e463,243.0,363.0,153.0,500,924,956,19.7,6
4,4,tomo_00e463,222.0,379.0,144.0,500,924,956,19.7,6
5,5,tomo_00e463,225.0,262.0,628.0,500,924,956,19.7,6
...,...,...,...,...,...,...,...,...,...,...
723,723,tomo_fb6ce6,-1.0,-1.0,-1.0,800,928,928,13.2,0
725,725,tomo_fc1665,-1.0,-1.0,-1.0,500,927,959,13.1,0
728,728,tomo_fc90fd,-1.0,-1.0,-1.0,500,1912,1847,6.5,0
733,733,tomo_fe85f6,-1.0,-1.0,-1.0,800,928,960,13.1,0


In [ ]:
# Create YOLO format label
# YOLO format: <class> <x_center> <y_center> <width> <height>
# Values are normalized to [0, 1]
# {"file_name": "0001.png", "objects": {"bbox": [[302.0, 109.0, 73.0, 52.0]], "categories": [0]}}
BBOXS_SIZE = 70
detections = {
    "images": [],  # list of all images in the dataset
    "annotations": [],  # list of all annotations in the dataset
    "categories": []  # list of all categories
}

trust = 2
rng = np.random.default_rng(42)

meta_train = []
meta_test = []
meta_val = []

size_empty = 3
source = []
destination = []
count_labl = 0

val_split = 0.1
test_split = 0.15

copy_files = True


def splittung(data, val_split=0.1, test_split=0.15):
    # test split
    size_det = len(data)

    test_split_len = int(size_det*test_split)

    test_tomos = np.random.choice(
        data, replace=False, size=test_split_len)

    train_tomos = []
    tomo=()
    [train_tomos.append(tomo) for tomo in data if tomo not in test_tomos]

    # Validation split
    size_train = len(train_tomos)
    val_split_len = int(size_train*val_split)

    val_tomos = np.random.choice(
        train_tomos, replace=False, size=val_split_len)
    train_tomos_new = []
    tomo=()
    [train_tomos_new.append(tomo)for tomo in train_tomos if tomo not in val_tomos]
    train_tomos = train_tomos_new
    return train_tomos, test_tomos, val_tomos


def transform_labels_to_coco(labels, trust):

    detections = {
        "images": [],  # list of all images in the dataset
        "annotations": [],  # list of all annotations in the dataset
        "categories": []  # list of all categories
    }

    detec_count = 0
    img_id_list = []

    for label in labels:

        for b in np.arange(int(label[2])-trust, int(label[2])+trust+1):
            if b < 0:
                continue
            width = float(BBOXS_SIZE)
            height = float(BBOXS_SIZE)
            if BBOXS_SIZE/2 > label[-2]:
                width = float((label[-2]-1)*2)
            if BBOXS_SIZE/2 > label[2]:
                height = float((label[2]-1)*2)

            img_id = int(f"{int(label[1].split('_')[1],16)}{b:04d}")
            img_name = f"{label[1]}_slice_{b:04d}.jpg"

            if img_id not in img_id_list:
                img_id_list.append(img_name)
                detections["images"].append({"id": img_id,
                                             "width": label[-3],
                                             "height": label[-4],
                                             "file_name": img_name
                                             })

            detections["annotations"].append({"id": detec_count,
                                              "image_id": img_id,  # the id of the image that the annotation belongs to
                                              "category_id": 0,  # the id of the category that the annotation belongs to
                                              "area": float(BBOXS_SIZE*BBOXS_SIZE),
                                              "bbox": [int(label[4]), int(label[3]), BBOXS_SIZE, BBOXS_SIZE],
                                              "iscrowd": 0
                                              })
            
            detec_count += 1

    detec_count = 0

    return detections


def get_coco_meta_file(detections, tomo_splits, coco_dir_path, source_path, copy_files=False):
    meta_val = {
    "images": [],  
    "annotations": [],  
    "categories": [{"id": 0, "name": "Motor"}]  
    }

    meta_train = {
    "images": [],  
    "annotations": [],  
    "categories": [{"id": 0, "name": "Motor"}]  
    }

    meta_test = {
    "images": [],  
    "annotations": [],  
    "categories": [{"id": 0, "name": "Motor"}]  
    }


    train_split, test_split, val_split = tomo_splits
    train_id = [int(i.split('_')[1], 16) for i in train_split]
    test_id = [int(i.split('_')[1], 16) for i in test_split]
    val_id = [int(i.split('_')[1], 16) for i in val_split]



    for image in detections["images"]:
        name=image["file_name"]
        tomo_name=f"{name.split('_')[0]}_{name.split('_')[1]}"
        if tomo_name in train_split:
            meta_train["images"].append(image)
        if tomo_name in test_split:
            meta_test["images"].append(image)
        if tomo_name in val_split:
            meta_val["images"].append(image)

    for label in detections["annotations"]:
        if int(str(label["image_id"])[:-4]) in train_id:
            meta_train["annotations"].append(label)
        elif int(str(label["image_id"])[:-4]) in test_id:
            meta_test["annotations"].append(label)
        elif int(str(label["image_id"])[:-4]) in val_id:
            meta_val["annotations"].append(label)

    # collect files to coco image folder
    if copy_files:
        for image in detections["images"]:
            split_list=image["file_name"].split("_")
            destination = coco_dir_path+image["file_name"]
            source = source_path+f"{split_list[0]}_{split_list[1]}/{split_list[2]}_{split_list[-1]}"
            if not os.path.isdir(os.path.dirname(destination)):
                os.mkdir(os.path.dirname(destination))
            shutil.copyfile(source, destination)

    meta_path="coco/annotations/"
    with open(meta_path+"train_labels.json", 'w') as json_output:
        json.dump(meta_train, json_output)
        json_output.write('\n')

    with open(meta_path+"test_labels.json", 'w') as json_output:
        json.dump(meta_test, json_output)
        json_output.write('\n')


    with open(meta_path+"val_labels.json", 'w') as json_output:
        json.dump(meta_val, json_output)
        json_output.write('\n')


tomot_with_motor_list = df[df["Number of motors"] > 0]["tomo_id"].unique()

data = splittung(tomot_with_motor_list)

detections = transform_labels_to_coco(labels, trust)
dest_path = "coco/images/"
source_path = "train/"

get_coco_meta_file(detections, data, dest_path, source_path, copy_files=False)

"""

<dataset_dir>/
    images/
        <imagename0>.<ext>
        <imagename1>.<ext>
        <imagename2>.<ext>
        ...
    annotations/
        train_labels.json
        val_labels.json
        test_labels.json
        ...

{
    "info": info,   
    "licenses": [license], 
    "images": [image],  # list of all images in the dataset
    "annotations": [annotation],  # list of all annotations in the dataset
    "categories": [category]  # list of all categories
}

where:

annotation = {
    "id": int, 
    "image_id": int,  # the id of the image that the annotation belongs to
    "category_id": int,  # the id of the category that the annotation belongs to
    "segmentation": RLE or [polygon], 
    "area": float, 
    "bbox": [x,y,width,height], 
    "iscrowd": int,  # 0 or 1,
}

category = {
    "id": int, 
    "name": str, 
    "supercategory": str,
}



image = {
    "id": int, 
    "width": int, 
    "height": int, 
    "file_name": str, 
    "license": int,  # the id of the license
    "date_captured": datetime,
}"""

# write to jason for hugginface

9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11


'\n\n<dataset_dir>/\n    images/\n        <imagename0>.<ext>\n        <imagename1>.<ext>\n        <imagename2>.<ext>\n        ...\n    annotations/\n        train_labels.json\n        val_labels.json\n        test_labels.json\n        ...\n\n{\n    "info": info,   \n    "licenses": [license], \n    "images": [image],  # list of all images in the dataset\n    "annotations": [annotation],  # list of all annotations in the dataset\n    "categories": [category]  # list of all categories\n}\n\nwhere:\n\nannotation = {\n    "id": int, \n    "image_id": int,  # the id of the image that the annotation belongs to\n    "category_id": int,  # the id of the category that the annotation belongs to\n    "segmentation": RLE or [polygon], \n    "area": float, \n    "bbox": [x,y,width,height], \n    "iscrowd": int,  # 0 or 1,\n}\n\ncategory = {\n    "id": int, \n    "name": str, \n    "supercategory": str,\n}\n\n\n\nimage = {\n    "id": int, \n    "width": int, \n    "height": int, \n    "file_name": s

In [ ]:
print()

In [15]:
detections["annotations"][-1]

{'id': 3153,
 'image_id': 16732252114,
 'category_id': 0,
 'area': 4900.0,
 'bbox': [678, 816, 70, 70],
 'iscrowd': 0}